# Open AIs Agent Framework

https://openai.com/index/new-tools-for-building-agents/

## Sessions

In [1]:
from agents import set_trace_processors
set_trace_processors([]) # disable trace processors for this notebook

In [2]:

import os
from agents import Agent, OpenAIChatCompletionsModel, Runner, SQLiteSession
from openai import AsyncOpenAI, AsyncAzureOpenAI


endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

client = AsyncAzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-08-01-preview")

model = OpenAIChatCompletionsModel( 
    model="gemma-3-4b-it-qat",
    openai_client=client
)


session_id = "conversation_123"
session = SQLiteSession(session_id)

haiku_agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=model
)

result = await Runner.run(haiku_agent, "Tell me about recursion in programming.", session=session)
print(result.final_output)



Function calls itself,  
Solving problems in layers,  
Base case ends the flow.  


In [3]:
result = await Runner.run(haiku_agent, "Translate this into german.", session=session)
print(result.final_output)


Funktion ruft sich,  
Löst Probleme in Schichten,  
Basisfall beendet.  


## Tool Calls

In [4]:
from pydantic import BaseModel

from agents import Agent, Runner, function_tool


class Weather(BaseModel):
    city: str
    temperature_range: str
    conditions: str


@function_tool
def get_weather(city: str) -> Weather:
    print("[debug] get_weather called")
    return Weather(city=city, temperature_range="14-20C", conditions="Sunny with wind.")


haiku_agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=model,
    tools=[get_weather]
)

result = await Runner.run(haiku_agent, input="What's the weather in Tokyo?")
print(result.final_output)


[debug] get_weather called
Tokyo's weather bright,  
Sunny skies and gentle breeze,  
Temperature's just right.
